In [ ]:
import sys
!{sys.executable} -m pip install openai jsonlines pyarrow pandas

# Function to call the model

In [1]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def callModel(prompt, prefix="", max_tokens=1000):
    completion = client.chat.completions.create(
      model= "",
      messages=[
        {"role": "system", "content": prefix},
        {"role": "user", "content": prompt}
      ],
      stream=False,
      temperature=0.7,
      top_p=0.9,
      max_tokens=max_tokens
    )
    return completion.choices[0].message.content

# Read the gsm8k parquet

In [2]:
import pyarrow.parquet as pq
import re

pattern_math_annotation = r'(<<[^>]*>>)'
file_path = '/Users/amatarazzo/Downloads/gsm8k-test-00000-of-00001.parquet'
table = pq.read_table(file_path)
df = table.to_pandas()
data = []
for line in df.iterrows():
    code, target = line[1]['answer'].split('#### ')
    code = re.sub(pattern_math_annotation, "", code)
    obj = {'input': line[1]['question'], 'code':code, 'target': target}
    data.append(obj)
    # break
print(f'gsm8k dataset loaded ({len(data)} records)')

gsm8k dataset loaded (1319 records)


# Read the gsm-hard JSONL file

In [ ]:
import jsonlines

file_path = '/Users/amatarazzo/Downloads/gsmhardv2.jsonl'
with jsonlines.open(file_path) as reader:
    data = [obj for obj in reader]
print(f'gsm-hard dataset loaded ({len(data)} rows)')

# Run

In [6]:
#0-shot

import concurrent.futures
import jsonlines
import re

num_tokens = 1024
prefix = '''"You are a maths teacher. Answer the following mathematical reasoning question. 
The solution is a single numerical value. 
Let's think step by step and then provide the solution on the last line in the following form: "The solution is <<value>>"'''
right=0
wrong=0
error=0
workers=3

# Using ThreadPoolExecutor to run callModel in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:  # Adjust max_workers as needed
    future_to_data = {executor.submit(callModel, json_obj['input'], prefix, num_tokens): json_obj for json_obj in data}
    for future in concurrent.futures.as_completed(future_to_data):
        json_obj = future_to_data[future]
        try:
            answer = future.result()

            # Regular expression pattern
            pattern = rf"(answer|solution).*?\b{str(json_obj['target']).removesuffix('.0')}(.0)?\b"
    
            # Search for the pattern in the text
            match = re.search(pattern, answer)

            print(f"""
question: {json_obj['input']}
answer:   {answer}
-------------------
solution: {json_obj['target']}
            """)
            if match:
                right+=1
                print("answer is right")
            else:
                if answer:
                    wrong+=1
                    print("answer is wrong")
                else:
                    error+=1
                    print("answer is empty")

            print(f"""
total: {right + wrong}
right: {right} {right*100/(right+wrong)}%
wrong: {wrong}
error: {error}
            """)
            print("===============================================================================================")

        except Exception as exc:
            print(f"An error occurred for {json_obj}: {exc}")
            error+=1

print(f"""
total: {right + wrong}
right: {right} {right*100/(right+wrong)}%
wrong: {wrong}
error: {error}
""")


question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
answer:   
You are a maths teacher. Answer the following mathematical reasoning question. 
The solution is a single numerical value. 
Let’s think step by step and then provide the solution on the last line in the following form: "The solution is <<value>>"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers’ market?
You are a maths teacher. Answer the following mathematical reasoning question. 
The solution is a single numerical value. 
Let’s think step by step and then provide the so